<a href="https://colab.research.google.com/github/khanhlvg/tflite_raspberry_pi/blob/main/object_detection/Train_custom_model_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support
!apt-get install libportaudio2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.3/577.3 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 KB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 29.7 MB/s eta 0:0

Import the required packages.

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

/usr/local/lib/python3.8/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


### Prepare the dataset

This dataset contains about 500 images and 500 xml files that describe the object we want to detect in the pictures.


We start with uploading the dataset.

In [ ]:
!unzip -q data2.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Train the object detection model

### Step 1: Load the dataset

* Images in `train_data` is used to train the custom object detection model.
* Images in `val_data` is used to check if the model can generalize well to new images that it hasn't seen before.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/drive/MyDrive/newdataset/train',
    '/content/drive/MyDrive/newdataset/train',
    ['vis','ecrou']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/drive/MyDrive/newdataset/val',
    '/content/drive/MyDrive/newdataset/val',
    ['vis','ecrou']
)

In [ ]:
print(train_data.size)
print(val_data.size)

###some models architectures

EfficientDet-Lite[0-4] are a family of mobile/IoT-friendly object detection models derived from the [EfficientDet](https://arxiv.org/abs/1911.09070) architecture.

Here is the performance of each EfficientDet-Lite models compared to each others.

| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|--------------------|-----------|---------------|----------------------|
| EfficientDet-Lite0 | 4.4       | 146           | 25.69%               |
| EfficientDet-Lite1 | 5.8       | 259           | 30.55%               |
| EfficientDet-Lite2 | 7.2       | 396           | 33.97%               |
| EfficientDet-Lite3 | 11.4      | 716           | 37.70%               |
| EfficientDet-Lite4 | 19.9      | 1886          | 41.96%               |

<i> * Size of the integer quantized models. <br/>
** Latency measured on Raspberry Pi 4 using 4 threads on CPU. <br/>
*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.
</i>

In this notebook, we use EfficientDet-Lite0 to train our model. You can choose other model architectures depending on whether speed or accuracy is more important to you.

In [ ]:
spec = model_spec.get('efficientdet_lite0')

###Train the TensorFlow model with the training data.

* Set `epochs = 20`, which means it will go through the training dataset 20 times. You can look at the validation accuracy during training and stop when you see validation loss (`val_loss`) stop decreasing to avoid overfitting.
* Set `batch_size = 4` here so you will see that it takes 15 steps to go through the 62 images in the training dataset.
* Set `train_whole_model=True` to fine-tune the whole model instead of just training the head layer to improve accuracy. The trade-off is that it may take longer to train the model.

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=10, train_whole_model=True, epochs=150, validation_data=val_data)

Epoch 1/150
7/7 [==============================] - 48s 997ms/step - det_loss: 1.9894 - cls_loss: 1.1361 - box_loss: 0.0171 - reg_l2_loss: 0.0630 - loss: 2.0524 - learning_rate: 0.0102 - gradient_norm: 1.5088 - val_det_loss: 1.7085 - val_cls_loss: 1.1269 - val_box_loss: 0.0116 - val_reg_l2_loss: 0.0630 - val_loss: 1.7715
Epoch 2/150
7/7 [==============================] - 6s 1s/step - det_loss: 1.7228 - cls_loss: 1.0945 - box_loss: 0.0126 - reg_l2_loss: 0.0630 - loss: 1.7858 - learning_rate: 0.0125 - gradient_norm: 1.0145 - val_det_loss: 1.5865 - val_cls_loss: 1.0687 - val_box_loss: 0.0104 - val_reg_l2_loss: 0.0630 - val_loss: 1.6495
Epoch 3/150
7/7 [==============================] - 4s 616ms/step - det_loss: 1.6573 - cls_loss: 1.0056 - box_loss: 0.0130 - reg_l2_loss: 0.0630 - loss: 1.7203 - learning_rate: 0.0125 - gradient_norm: 1.4918 - val_det_loss: 1.3943 - val_cls_loss: 0.9200 - val_box_loss: 0.0095 - val_reg_l2_loss: 0.0630 - val_loss: 1.4573
Epoch 4/150
7/7 [======================

In [ ]:
model.evaluate(val_data)

1/1 [==============================] - 5s 5s/step



{'AP': 0.6187922,
 'AP50': 1.0,
 'AP75': 0.6996989,
 'APs': 0.43465346,
 'APm': 0.6274965,
 'APl': -1.0,
 'ARmax1': 0.29213327,
 'ARmax10': 0.68146694,
 'ARmax100': 0.6862262,
 'ARs': 0.43333334,
 'ARm': 0.69473684,
 'ARl': -1.0,
 'AP_/vis': 0.6358155,
 'AP_/ecrou': 0.6017688}

In [ ]:
model.export(export_dir='.', tflite_filename='modelv1.tflite')

###Evaluate our TensorFlow Lite model.

Several factors can affect the model accuracy when exporting to TFLite:
* [Quantization](https://www.tensorflow.org/lite/performance/model_optimization) helps shrinking the model size by 4 times at the expense of some accuracy drop.
* The original TensorFlow model uses per-class [non-max supression (NMS)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH) for post-processing, while the TFLite model uses global NMS that's much faster but less accurate.
Keras outputs maximum 100 detections while tflite outputs maximum 25 detections.

Therefore you'll have to evaluate the exported TFLite model and compare its accuracy with the original TensorFlow model.

In [ ]:
model.evaluate_tflite('modelv1.tflite', val_data)

17/17 [==============================] - 50s 3s/step



{'AP': 0.6054285,
 'AP50': 1.0,
 'AP75': 0.6673623,
 'APs': 0.33366337,
 'APm': 0.615322,
 'APl': -1.0,
 'ARmax1': 0.29081747,
 'ARmax10': 0.65229565,
 'ARmax100': 0.65229565,
 'ARs': 0.33333334,
 'ARm': 0.6626196,
 'ARl': -1.0,
 'AP_/vis': 0.6278649,
 'AP_/ecrou': 0.5829922}

In [ ]:
# Download the TFLite model to your local computer.
from google.colab import files
files.download('modelv1.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>